In [1]:
%pip install torch


--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/Library/

In [2]:
%pip install transformers

--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/Library/

In [1]:
import json
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [2]:
with open('maysey-2.json', 'r') as f:
    data = json.load(f)

messages = []
for key in data:
    messages.append(data[key])

text = '\n'.join(messages)

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')  # specify gpt2-large model
model = GPT2LMHeadModel.from_pretrained('gpt2-large') 

In [4]:
encoded_text = tokenizer.encode(text, max_length=200000,truncation=True)

In [5]:
file_path = './maysey-2.json'
dataset = TextDataset(tokenizer=tokenizer, file_path=file_path, block_size=512)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [6]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [7]:
training_args = TrainingArguments(
    output_dir='./results',  # Output directory
    overwrite_output_dir=True,  # Overwrite the content of the output directory
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=16,  # Batch size per GPU
    gradient_accumulation_steps=32,  # Number of updates steps to accumulate before performing a backward/update pass
    learning_rate=5e-5,  # Learning rate
    warmup_steps=500,  # Number of warmup steps for learning rate scheduler
    save_steps=10_000,  # Number of steps between checkpoint saves
    save_total_limit=2,  # Limit the total amount of checkpoints that are saved
)


In [8]:
trainer = Trainer(
    model=model,  # The instantiated 🤗 Transformers model to be trained
    args=training_args,  # Training arguments, defined above
    train_dataset=dataset,  # Training dataset
    data_collator=data_collator,  # Data collator
)

In [9]:
trainer.train()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/3 [00:00<?, ?it/s]

{'train_runtime': 7417.0489, 'train_samples_per_second': 0.203, 'train_steps_per_second': 0.0, 'train_loss': 3.4749857584635415, 'epoch': 3.0}


TrainOutput(global_step=3, training_loss=3.4749857584635415, metrics={'train_runtime': 7417.0489, 'train_samples_per_second': 0.203, 'train_steps_per_second': 0.0, 'train_loss': 3.4749857584635415, 'epoch': 3.0})

In [10]:
trainer.save_model('./cassie')

In [1]:
import discord
from discord.ext import commands
import json
import os
from transformers import AutoTokenizer, GPT2LMHeadModel

tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    
model = GPT2LMHeadModel.from_pretrained('./cassie')

# Set pad token id
model.config.pad_token_id = tokenizer.pad_token_id

In [2]:
async def main():
    # Initialize the Discord bot
    intents = discord.Intents.default()
    intents.members = True
    intents.message_content = True 
    bot = commands.Bot(command_prefix='!',intents=intents)

    @bot.event
    async def on_ready():
        print(f'Logged in as {bot.user} (ID: {bot.user.id})')

    @bot.command()
    async def dosomething(ctx):
        await ctx.send("I did something")
    
    async def generate_response(message):
         input_ids = tokenizer.encode(message, return_tensors='pt')
         bot_output = model.generate(input_ids, max_length=50000, do_sample=True)
         bot_response = tokenizer.decode(bot_output[0], skip_special_tokens=True)
         return bot_response
    
    @bot.command()
    async def chat(ctx):
     message = ctx.message
     if message.author == bot.user:
        return

    # Generate a response using the model
     print(message.content)
     response = await generate_response(message.content)
     print(response)

    # Split the bot response into chunks of 2000 characters or less
     response_chunks = [response[i:i+2000] for i in range(0, len(response), 2000)]

     try:
        # Send each chunk as a separate message
      for chunk in response_chunks:
            await ctx.send(chunk)
     except Exception as e:
        print(e)


    # Start the bot
    
    await bot.start('MTA4ODMyNTYyNDg5NDA4MzA4Mg.GG1Dkv.zWjWxkw3mwTEOxFJ0-3WWkgCyeidH7bla1DTzo')

await main()

Logged in as C.A.S.S.I.E#6203 (ID: 1088325624894083082)
!chat konichiwa


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


!chat konichiwa)

(hmm, how could I not, since I am from my home)

hmm, so my dad just disappeared?

*smirks*

(I'll give him
!chat konichiwa
!chat konichiwa [22:08:08] <the_monster> yes, i'm sure that's what I'm thinking [22:08:13] <shadowbiell> i need to stop taking things so often [
!chat bts
!chat btsbcs5bfa85a4b36bf2ddbeb145518c2bc4bc48bbf0aa50> /m/dw3r/reddit/comments/51y55u
!chat jungkook
!chat jungkook3w and sepranigans

▶ Leader 02/05/15 (Tue) 27:15:38 ID: 0844ba (1) No.453738 >>454755 >>534
!chat hi
!chat hi@zombie_wars.org > > I am not very experienced in using zombie-wars and have not seen any work with it. I will work with any team I see and may be able to fix it on
!chat I deserve to reward myself with special treats and activities.
!chat I deserve to reward myself with special treats and activities. If you are enjoying this activity, please send suggestions by posting them in the reddit post: http://www.reddit.com/r/CourierGifts/comments/2
!chat post a gif
!chat post a g